# Simulación de una Cadena de Markov en un Proceso de Acceso Aleatorio (RACH)

# Introducción
Los modelos probabilísticos son herramientas fundamentales en la estadística y la ciencia de datos para representar procesos inciertos o aleatorios. Uno de los más utilizados en teoría de procesos estocásticos es la cadena de Markov, especialmente útil cuando se desea modelar una secuencia de eventos donde la probabilidad de transición entre estados depende únicamente del estado actual y no del historial previo. Cuando estos modelos incluyen estados absorbentes, representan escenarios donde una vez alcanzado cierto estado, el sistema ya no cambia más.

En este proyecto, se analiza un modelo basado en una cadena de Markov finita con estados absorbentes que representa un proceso de hasta tres intentos por parte de un usuario para lograr un objetivo (por ejemplo, una autenticación, completar una compra, o transmitir un paquete en redes). En cada intento, el usuario tiene una probabilidad constante de éxito p = 0.6. Si no logra el éxito en los tres intentos permitidos, el proceso se considera como fallido.

El objetivo del estudio es doble:

Calcular de forma analítica las probabilidades de éxito y fracaso, así como el número esperado de intentos por usuario, utilizando teoría de cadenas de Markov.
Realizar una simulación Monte Carlo con un millón de usuarios para estimar los mismos valores empíricamente y comparar los resultados con los obtenidos teóricamente.

In [11]:
import numpy as np

# =========================
# PARÁMETROS DEL PROCESO
# =========================

# Probabilidad de éxito en un intento
p_exito = 0.60
p_fallo = 1 - p_exito

# Estados:
# S0, S1, S2 → transitorios (intentos)
# S3 → absorbente (éxito)
# S4 → absorbente (fracaso)

# Construcción directa de matrices Q y R
Q = np.array([
    [0, p_fallo, 0],
    [0, 0, p_fallo],
    [0, 0, 0]
])

R = np.array([
    [p_exito, 0],
    [p_exito, 0],
    [p_exito, p_fallo]
])

# =========================
# ANÁLISIS MATEMÁTICO
# =========================

I = np.identity(3)
N = np.linalg.inv(I - Q)        # Matriz fundamental
absorcion = N @ R               # Probabilidades de absorción

# Resultados desde el estado inicial S0
prob_exito_teorica = absorcion[0, 0]
prob_fallo_teorica = absorcion[0, 1]
intentos_teoricos = N[0] @ np.ones(3)

# =========================
# SIMULACIÓN MONTE CARLO
# =========================

rng = np.random.default_rng(seed=123)
num_pruebas = 1_000_000
total_intentos = 0
exitos = 0
fracasos = 0

# Función de intento por usuario
def intento_usuario(prob_exito: float) -> int:
    for i in range(1, 4):
        if rng.random() < prob_exito:
            return i  # Éxito en i-ésimo intento
    return 0  # Fracaso después de 3 intentos

# Ejecutar simulación para N usuarios
for _ in range(num_pruebas):
    resultado = intento_usuario(p_exito)
    if resultado > 0:
        exitos += 1
        total_intentos += resultado
    else:
        fracasos += 1
        total_intentos += 3

# =========================
# RESULTADOS FINALES
# =========================

print("=== CÁLCULO TEÓRICO ===")
print(f"Probabilidad de Éxito   : {prob_exito_teorica:.6f}")
print(f"Probabilidad de Fracaso : {prob_fallo_teorica:.6f}")
print(f"Intentos Esperados      : {intentos_teoricos:.6f}")

print("\n=== RESULTADOS SIMULADOS ===")
print(f"Probabilidad de Éxito   : {exitos / num_pruebas:.6f}")
print(f"Probabilidad de Fracaso : {fracasos / num_pruebas:.6f}")
print(f"Intentos Promedio       : {total_intentos / num_pruebas:.6f}")


=== CÁLCULO TEÓRICO ===
Probabilidad de Éxito   : 0.936000
Probabilidad de Fracaso : 0.064000
Intentos Esperados      : 1.560000

=== RESULTADOS SIMULADOS ===
Probabilidad de Éxito   : 0.936063
Probabilidad de Fracaso : 0.063937
Intentos Promedio       : 1.559885


# Análisis del Código
El código implementado para este estudio se divide en dos partes principales: el análisis analítico del proceso y la simulación por Monte Carlo.

Parte 1: Cálculo Teórico con Cadena de Markov

Se define primero la matriz de transición P, de tamaño 5x5, que contiene las probabilidades de moverse entre estados. Luego se extraen las submatrices:

Q: Submatriz de tamaño 3x3 que representa las transiciones entre los estados transitorios (intentos).
R: Submatriz de tamaño 3x2 que contiene las probabilidades de transición desde los estados transitorios hacia los estados absorbentes (éxito y fracaso).
Se calcula la matriz fundamental N aplicando la fórmula matricial N = (I - Q)^-1. Esta matriz contiene en su fila 0 el número esperado de veces que el sistema pasa por cada uno de los estados transitorios si comienza en el primer intento.

Posteriormente se calcula B = N @ R, que contiene la probabilidad de terminar en éxito o fracaso, partiendo de cada uno de los estados transitorios. El resultado más relevante es la primera fila de B, que indica la probabilidad de éxito y de fracaso si el usuario comienza desde el estado inicial (intento 1).

Por último, se calcula el número esperado de intentos como t = N @ 1, donde 1 es un vector columna de unos. El primer valor de este vector representa el número promedio de intentos necesarios para que el proceso termine, comenzando desde el estado 0.

Parte 2: Simulación Monte Carlo

Se implementa una función que simula el proceso para un usuario. Esta función genera hasta tres números aleatorios y determina si alguno de ellos es menor que la probabilidad p, lo que representaría un intento exitoso. Si no se logra el éxito en los tres intentos, el proceso se clasifica como fracaso.

Se realiza esta simulación para un millón de usuarios, acumulando el número total de intentos, la cantidad de éxitos y la cantidad de fracasos. Con estos valores se calculan las métricas deseadas:

Probabilidad empírica de éxito: éxitos / total_usuarios
Probabilidad empírica de fracaso: fracasos / total_usuarios
Número promedio de intentos: total_intentos / total_usuarios
Esta parte del código es eficiente y simple, y utiliza el generador de números aleatorios de NumPy con semilla (seed) para garantizar que los resultados puedan reproducirse.



# Comparación de Resultados
Los valores simulados son prácticamente idénticos a los valores teóricos, con una diferencia mínima atribuible a la variabilidad inherente de una simulación aleatoria, incluso con un número tan alto como un millón de muestras.

Este nivel de coincidencia valida tanto el modelo teórico basado en la cadena de Markov como la implementación de la simulación. También demuestra que, para este tipo de procesos, las cadenas de Markov ofrecen una forma precisa y elegante de anticipar comportamientos sin necesidad de pruebas empíricas extensas.

# Conclusión
Este trabajo demuestra la utilidad y precisión de las cadenas de Markov con estados absorbentes para modelar procesos con intentos limitados y resultados binarios (éxito o fracaso). El análisis matemático permitió obtener de forma exacta las probabilidades de éxito y fracaso, así como el número esperado de intentos, sin necesidad de experimentar.

Por otro lado, la simulación Monte Carlo sirvió como una herramienta práctica para validar los resultados analíticos y verificar el funcionamiento del modelo desde una perspectiva empírica. A pesar de ser un proceso aleatorio, la alta coincidencia entre los resultados demuestra que el número de simulaciones fue adecuado y que el sistema es confiable.

Este tipo de modelado puede aplicarse en múltiples contextos reales como:

Sistemas de autenticación con intentos limitados.
Protocolos de retransmisión de datos en redes.
Juegos de azar o experimentos de tipo Bernoulli con repetición limitada.
Procesos de toma de decisiones donde los recursos (intentos) son finitos.
En resumen, la combinación de teoría de Markov con simulación computacional es una herramienta poderosa para comprender y predecir el comportamiento de sistemas complejos, especialmente cuando estos tienen una estructura secuencial y probabilística claramente definida.